In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import csv
import os
import pandas as pd
import numpy as np
import random
import copy
from datetime import datetime

# Get Raw Data

#Get checkin data from Review Data

In [3]:
# JSON 파일 경로
json_file_path = '/content/drive/MyDrive/dataset/yelp_dataset/review.json'

# CSV 파일 경로
csv_file_path = '/content/checkin_raw.csv'

# CSV 파일을 쓰기 모드로 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    # CSV 라이터 생성
    csv_writer = csv.writer(csv_file)

    # CSV 파일 헤더 작성
    csv_writer.writerow(['user_id', 'business_id'])

    # JSON 파일을 한 줄씩 읽어서 처리
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            data = json.loads(line)  # JSON 데이터 파싱
            # user_id와 business_id 추출
            user_id = data['user_id']
            business_id = data['business_id']

            # CSV 파일에 데이터 작성
            csv_writer.writerow([user_id, business_id])

print("checkin_data 생성 완료.")


checkin_data 생성 완료.


In [4]:
"""
데이터 개수(row 수) 확인
"""

# CSV 파일 경로
csv_file_path = '/content/checkin_raw.csv'

row_count = 0

# CSV 파일을 읽기 모드로 열고 행 수를 센다
with open(csv_file_path, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # 헤더 행을 건너뜁니다.
    for row in csv_reader:
        row_count += 1

print(f"CSV 파일의 행 수: {row_count}")

CSV 파일의 행 수: 6990280


### Business_info Data 생성
Business 데이터로 가게의 위치를 담은 데이터(business_id,location, city)를 만든다 <br>
원하는 데이터 정보 추가,삭제 : 13, 21 수정

In [5]:
# JSON 파일 경로
json_file_path = '/content/drive/MyDrive/dataset/yelp_dataset/business.json'

# CSV 파일 경로
csv_file_path = '/content/business_info_raw.csv'

# CSV 파일을 쓰기 모드로 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    # CSV 라이터 생성
    csv_writer = csv.writer(csv_file)

    # CSV 파일 헤더 작성
    csv_writer.writerow(['business_id','latitude ','longitute', 'city'])

    # business_id 를 int형으로 변환
    idx = 0

    # JSON 파일을 한 줄씩 읽어서 처리
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            data = json.loads(line)  # JSON 데이터 파싱

            # user_id와 business_id 추출
            business_id = data['business_id']
            latitude = data['latitude']
            longitude = data['longitude']
            city = data['city']

            # CSV 파일에 데이터 작성
            csv_writer.writerow([business_id, latitude, longitude, city])
            idx += 1

print("CSV 파일 생성 완료.")


CSV 파일 생성 완료.


In [6]:
"""
데이터 개수(row 수) 확인
"""
# CSV 파일 경로
csv_file_path = '/content/business_info_raw.csv'

row_count = 0

# CSV 파일을 읽기 모드로 열고 행 수를 센다
with open(csv_file_path, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # 헤더 행을 건너뜁니다.
    for row in csv_reader:
        row_count += 1

print(f"CSV 파일의 행 수: {row_count}")

CSV 파일의 행 수: 150346


## 사용자 방문데이터(위치포함) 생성
위에서 생성한 데이터 두개를 합친 방문데이터 생성 <br>
checkin + business_info => location과 city 확인<br>
Philadelphia 도시에 대한 데이터만 남김

In [7]:
# business_info.csv 파일 경로
business_info_file = '/content/business_info_raw.csv'

# checkin_data.csv 파일 경로
checkin_data_file = '/content/checkin_raw.csv'

# 새로운 CSV 파일 경로 (결과 저장용)
output_file = '/content/checkin_data_in_philadelphia.csv'

# business_id를 키로 한 dictionary 생성
business_location = {}
with open(business_info_file, 'r', newline='') as business_file:
    csv_reader = csv.reader(business_file)
    next(csv_reader)  # 헤더 행 건너뛰기
    for row in csv_reader:
        business_id, latitude, longitude, city = row[0], row[1], row[2], row[3].lower() #city : 소문자로 받음
        business_location[business_id] = (latitude, longitude, city)

count = 0 # 잘 다 들어갔는지 체크
mcount = 0 # 잘 안들어간거 있는지 체크
not_phila = 0

# 새로운 CSV 파일(output_file : checkin_data_in_philadelphia)을 작성하여 business_id, latitude, longitude를 추가
with open(checkin_data_file, 'r', newline='') as checkin_file, open(output_file, 'w', newline='') as output_csv:
    csv_reader = csv.reader(checkin_file)
    csv_writer = csv.writer(output_csv)

    # 헤더 행 작성
    header = next(csv_reader)
    header.extend(['business_id', 'latitude', 'longitude', 'city'])
    csv_writer.writerow(header)

    for row in csv_reader:
        business_id = row[1]
        if business_id in business_location:
            latitude, longitude, city = business_location[business_id]

            # city가 "philadelphia"인 경우에만 데이터를 CSV 파일에 작성
            if "philadelphia" in city: #philadelphia 에서 오타가 존재할 수 있기 때문에 "phila"만 검사
              row.extend([latitude, longitude, city])
              csv_writer.writerow(row)
              count += 1
            else:
              not_phila += 1
        else:
            # 해당 business_id가 business_info_raw.csv에 없을 경우, 스킵
            mcount += 1
            continue

print("CSV 파일에 위치 정보가 추가되었습니다.")
print(count, mcount, not_phila) #philadelphia, 에러, 다른도시


CSV 파일에 위치 정보가 추가되었습니다.
968050 0 6022230


### User ID, POI ID를 정수형으로 변경

In [8]:
# CSV 파일 경로
input_file = '/content/checkin_data_in_philadelphia.csv'
output_file = '/content/checkins.txt'

# CSV 파일을 읽고 데이터를 리스트로 저장
data = []
with open(input_file, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)  # 헤더 행
    #data.append(header)
    for row in csv_reader:
        data.append(row)

# user_id를 기준으로 데이터를 정렬
data.sort(key=lambda x: x[0])  # 여기서 0은 user_id 열을 가리킵니다. 0부터 시작하면 첫 번째 열입니다.

# user_id를 정수형으로 변환
idx = 0
before_user_id = data[0][0]
for i in data:
  if i[0] == before_user_id:
    i[0] = idx
  else:
    idx += 1
    before_user_id = i[0]
    i[0] = idx

# business_id 기준으로 데이터를 정렬
data.sort(key=lambda x: x[1])  # 여기서 0은 user_id 열을 가리킵니다. 0부터 시작하면 첫 번째 열입니다.
print(data[:3])

# business_id 정수형으로 변환
idx = 0
before_business_id = data[0][1]
for i in data:
  if i[1] == before_business_id:
    i[1] = idx
  else:
    idx += 1
    before_business_id = i[1]
    i[1] = idx

print(data[:3])

# 다시 유저 번호 기준으로 정렬
data.sort(key=lambda x: x[0])
print(data[:3])

# 정렬된 데이터를 새로운 파일에 저장
with open(output_file, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(data)

print("CSV 파일이 user_id를 기준으로 정렬되었고, 새로운 파일에 저장되었습니다.")

[[55363, '--OS_I7dnABrXvRCCuWOGQ', '40.0281603', '-75.0338026', 'philadelphia'], [65734, '--OS_I7dnABrXvRCCuWOGQ', '40.0281603', '-75.0338026', 'philadelphia'], [123557, '--OS_I7dnABrXvRCCuWOGQ', '40.0281603', '-75.0338026', 'philadelphia']]
[[55363, 0, '40.0281603', '-75.0338026', 'philadelphia'], [65734, 0, '40.0281603', '-75.0338026', 'philadelphia'], [123557, 0, '40.0281603', '-75.0338026', 'philadelphia']]
[[0, 1481, '40.0741872', '-75.2030839', 'philadelphia'], [0, 8854, '39.9544408', '-75.1704266', 'philadelphia'], [1, 7950, '39.9418368', '-75.1538396', 'philadelphia']]
CSV 파일이 user_id를 기준으로 정렬되었고, 새로운 파일에 저장되었습니다.


위 사용자들이 방문했던 POI들의 ID, 위치 정보를 담은 Business_info 생성

In [9]:
data.sort(key=lambda x: x[1])

column =['user_id', 'business_id', 'latitude', 'longitude', 'city']
df = pd.DataFrame(data, columns=column)
# 중복되지 않는 '키' 값을 찾고 필터링
result = df.drop_duplicates('business_id', keep='first')[['business_id', 'latitude', 'longitude', 'city']].values.tolist()
print(len(result))

with open('/content/business_info.csv', 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(result)

14586


## Test,Validation,Train Data 생성

In [16]:
print(len(data))
data[:5]

968050


[[55363, 0, '40.0281603', '-75.0338026', 'philadelphia'],
 [65734, 0, '40.0281603', '-75.0338026', 'philadelphia'],
 [123557, 0, '40.0281603', '-75.0338026', 'philadelphia'],
 [146960, 0, '40.0281603', '-75.0338026', 'philadelphia'],
 [246215, 0, '40.0281603', '-75.0338026', 'philadelphia']]

2차원 리스트 history list 생성  
[[1,123,35], : 1번째 유저 방문 기록  
[2,3,445,123,1222,123], : 2번째 유저 방문 기록  
...]

In [20]:
"""
train에 사용될 history 데이터 (2차원 리스트)
인덱스 번호 : user_Id
인덱스 value(list형식) : 해당 uesr가 방문한 business_id 리스트
"""
data.sort(key=lambda x: x[0]) # user id 기준으로 정렬

history_list = []
tmp = []
before_user_id = 0
for idx, i in enumerate(data):
  if i[0] == before_user_id:
    tmp.append(i[1])
  else:
    tmp = list(dict.fromkeys(tmp)) # dictionary key 특성을 이용한 중복제거
    if len(tmp) >= 10: # 중복이 제거된 방문 횟수가 10회가 넘는 유저만 append
      history_list.append(tmp)

    tmp = []
    tmp.append(i[1])
    before_user_id += 1

  if idx == len(data) - 1:
    if len(tmp) >= 10:
      history_list.append(tmp)

output_file = '/content/history_list.csv'
with open(output_file, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(history_list)

history_list

[[19,
  820,
  1122,
  1717,
  1835,
  4385,
  4478,
  5401,
  6170,
  6680,
  7995,
  8082,
  11025,
  11094,
  11748,
  13069,
  13511],
 [2295,
  3638,
  5779,
  6662,
  7589,
  8144,
  8622,
  8804,
  9622,
  10243,
  12783,
  12805,
  13071,
  13673,
  13795,
  14106,
  14444],
 [441,
  1193,
  2427,
  2915,
  7000,
  7878,
  8912,
  10010,
  10750,
  11025,
  12026,
  12201,
  13316],
 [1679, 2419, 3950, 4588, 6286, 6604, 8208, 9212, 9644, 10932, 11012],
 [7, 70, 3141, 3296, 5033, 5151, 6931, 10823, 11849, 13563, 14524],
 [1587, 2582, 3418, 5208, 7344, 7698, 10189, 13282, 13367, 13516],
 [1471, 1642, 6145, 6775, 7332, 9107, 9579, 9636, 9883, 10682, 11886, 12147],
 [132, 367, 667, 840, 3956, 4520, 6605, 9375, 11278, 12316, 13566],
 [132,
  1657,
  2348,
  6303,
  6320,
  6764,
  7649,
  7698,
  8900,
  10670,
  11027,
  11393,
  12603,
  13235],
 [550,
  1471,
  2076,
  2412,
  2508,
  3502,
  3726,
  5455,
  5462,
  6686,
  6758,
  8893,
  9842,
  10119,
  12016,
  12333,
  13367

In [21]:
business_info = pd.read_csv('/content/business_info.csv', names = ['business_id','latitude','longitude','city'])
print(business_info)
business_id_list = business_info['business_id'].tolist()
latitude_list = business_info['latitude'].tolist()
longitude_list = business_info['longitude'].tolist()

       business_id   latitude  longitude          city
0                0  40.028160 -75.033803  philadelphia
1                1  39.952584 -75.165222  philadelphia
2                2  39.967439 -75.175452  philadelphia
3                3  39.923048 -75.178078  philadelphia
4                4  39.948508 -75.161969  philadelphia
...            ...        ...        ...           ...
14581        14581  40.060970 -75.167412  philadelphia
14582        14582  40.022691 -75.208956  philadelphia
14583        14583  39.982937 -75.154732  philadelphia
14584        14584  39.936582 -75.162655  philadelphia
14585        14585  39.949183 -75.164777  philadelphia

[14586 rows x 4 columns]


In [22]:
"""
위 history_list를 train, validation, test 데이터셋으로 분리시킴
"""
train_ratio = 0.6
test_ratio = 0.2
validation_ratio = 0.2

train_dataset = []
test_dataset = []
validation_dataset = []

train_negative_dataset = []
test_negative_dataset = []
validation_negative_dataset = []


#아래의 결과를 csv 파일로 저장할 경로, 총 6개의 파일 생성
output_train = '/content/history_train.csv'
output_validation = '/content/history_validation.csv'
output_test = '/content/history_test.csv'

output_train_negative = '/content/history_train_negative.csv'
output_validation_negative = '/content/history_validation_negative.csv'
output_test_negative = '/content/history_test_negative.csv'


for user, history in enumerate(history_list):
  negative_list = [num for num in business_id_list if num not in history]

  ### 한 유저에 대해 train,validation,test dataset 생성
  total_samples = len(history) # 방문 횟수 저장
  random.shuffle(history) # 방문 기록 순서를 랜덤하게 섞어준다.

  train_size = int(total_samples * train_ratio)
  validation_size = int(total_samples * validation_ratio)

  list_train = history[:train_size]
  list_validation = history[train_size: (train_size + validation_size)]
  list_test = history[(train_size + validation_size):]


  train_dataset.append(list_train)
  validation_dataset.append(list_validation)
  test_dataset.append(list_test)

  total_samples = len(negative_list) # 방문 횟수 저장
  random.shuffle(negative_list) # 방문 기록 순서를 랜덤하게 섞어준다.

  train_size = int(total_samples * train_ratio)
  validation_size = int(total_samples * validation_ratio)

  negative_list_train = negative_list[:train_size]
  negative_list_validation = negative_list[train_size: (train_size + validation_size)]
  negative_list_test = negative_list[(train_size + validation_size):]

  train_negative_dataset.append(negative_list_train)
  validation_negative_dataset.append(negative_list_validation)
  test_negative_dataset.append(negative_list_test)

with open(output_train, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(train_dataset)
with open(output_validation, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(validation_dataset)
with open(output_test, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(test_dataset)

with open(output_train_negative, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(train_negative_dataset)
with open(output_validation_negative, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(validation_negative_dataset)
with open(output_test_negative, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(test_negative_dataset)